In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px
import platform
from datetime import datetime,timedelta


# 운영체제별 한글 폰트 설정
if platform.system() == 'Windows': # Windows 환경 폰트 설정
    plt.rc('font', family='Malgun Gothic')

plt.rc('axes', unicode_minus=False) # 마이너스 폰트 설정


# 글씨 선명하게 출력하는 설정
%config InlineBackend.figure_format = 'retina'




# dataset

In [2]:
df_1 = pd.read_csv('data/국립자연휴양림 예약 정보(200101~200824).csv')
df_2 = pd.read_csv('data/국립자연휴양림 예약 정보(201014~210322).csv')
df_3 = pd.read_csv('data/국립자연휴양림 예약 정보(210329~210923).csv')
df_4 = pd.read_csv('data/국립자연휴양림 예약 정보(210924~220314).csv')

df_info = pd.read_csv('data/산림청 국립자연휴양림관리소_국립자연휴양림 시설관련 정보_20220906.csv')
df_stat = pd.read_csv('data/산림청 국립자연휴양림관리소_국립자연휴양림 이용객 정산관련 데이터_20200630.csv')
df_describe = pd.read_csv('data/산림청 국립자연휴양림관리소_국립자연휴양림 홍보_20220906.csv')

# pre-processing

In [3]:
df_1 = df_1[['기관ID','기관이름','숙박일자']]
df_1 = df_1.rename(columns={'기관ID':'id','기관이름':'name','숙박일자':'date'})
df_1['id'] = df_1['id'].str.replace("'","")
df_1['name'] = df_1['name'].str.replace("'","")
df_1['date'] = df_1['date'].str.replace("'","")
df_1 = df_1[df_1['date'] != '2(24.7㎡×2실)']  
df_1['date'] = df_1['date'].astype('datetime64')

df_1 = df_1[['date','id','name']]

df_2 = df_2[['기관ID','기관이름','숙박일자']]
df_2 = df_2.rename(columns={'기관ID':'id','기관이름':'name','숙박일자':'date'})
df_2['date'] = df_2['date'].astype('str').astype('datetime64')

df_2 = df_2[['date','id','name']]

df_3 = df_3[['INSTT_ID','INSTT_NM','STNG_DT']]
df_3 = df_3.rename(columns={'INSTT_ID':'id','INSTT_NM':'name','STNG_DT':'date'})
df_3['date'] = df_3['date'].astype('str').astype('datetime64')

df_3 = df_3[['date','id','name']]

df_4 = df_4[['기관아이디','기관이름','숙박일자']]
df_4 = df_4.rename(columns={'기관아이디':'id','기관이름':'name','숙박일자':'date'})
df_4['date'] = df_4['date'].astype('str').astype('datetime64')

df_4 = df_4[['date','id','name']]

# 총 예약 데이터
df = pd.concat([df_1,df_2,df_3,df_4,])

In [4]:
df.isnull().sum()

date    0
id      0
name    0
dtype: int64

In [5]:
df.id = df.id.astype('str').apply(lambda x : x[-3:]).astype('int64')

In [6]:
df['month'] = df['date'].dt.month
df['weekday'] = df['date'].dt.weekday

In [7]:
# 휴양림 별 예약건수
df_id = df.groupby(['id'],as_index=False).agg(resv_count = ('id','count'))

In [8]:
df_info = df_info.drop_duplicates(['아이디'])
df_info = df_info[['아이디','기관명','주소','상품위치위도','상품위치경도']]
df_info = df_info.rename(columns={'아이디':'id','기관명':'name','주소':'address','상품위치위도':'lat','상품위치경도':'lon',})

In [9]:
df_id = pd.merge(df_id,df_info,on='id')

In [10]:
df_stat = df_stat[['휴양림명','총 가동률','주중 가동률','주말 가동률','합계 금액']]
df_stat = df_stat.rename(columns={'휴양림명':'name','총 가동률':'rate','주중 가동률':'wd_rate','주말 가동률':'we_rate','합계 금액':'revenue'})

In [11]:
df_id = pd.merge(df_id,df_stat,on='name')

In [12]:
df_describe = df_describe[['기관명','최대수용인원','기관설명']]
df_describe = df_describe.rename(columns={'기관명':'name','최대수용인원':'capacity','기관설명':'describe'})

In [13]:
df_id = pd.merge(df_id,df_describe,on='name')

In [14]:
df_id.columns 

Index(['id', 'resv_count', 'name', 'address', 'lat', 'lon', 'rate', 'wd_rate',
       'we_rate', 'revenue', 'capacity', 'describe'],
      dtype='object')

In [17]:
df_id = df_id[['id', 'name', 'address', 'lat', 'lon', 'capacity', 'rate', 'wd_rate',
       'we_rate', 'resv_count', 'revenue', 'describe']]

In [18]:
df_id

,id,name,address,lat,lon,capacity,rate,wd_rate,we_rate,resv_count,revenue,describe
0,101,유명산 자연휴양림,(12473) 경기 가평군 설악면 유명산길 79-53,127.4821423,37.59010292,3000,79.258,76.503,84.123,33716,187641300,자생식물원을 품은 수도권의 숲속 쉼터 유명산 자연휴양림
1,102,용대 자연휴양림,(24605) 강원 인제군 북면 연화동길 7,128.3434525,38.23701579,800,75.931,71.742,83.579,10532,57035200,"오염되지 않은 자연 속 휴식 공간, 용대 자연휴양림"
2,103,산음 자연휴양림,(12527) 경기 양평군 단월면 고북길 347,127.5689189,37.59769924,2000,71.754,65.420,83.439,21234,115218600,자연 그대로의 모습이 보존된 산음 자연휴양림
3,104,아세안 자연휴양림,(11518) 경기 양주시 백석읍 기산로 472,126.9426123,37.77297894,200,83.821,82.296,86.711,10632,94209900,아세안 10개국의 문화 체험을 즐길 수 있는 이색 휴양림 아세안 자연휴양림
4,105,신불산 자연휴양림,(44909) 울산 울주군 상북면 청수골길 175,129.0235555,35.54455149,1000,56.574,48.306,70.961,12224,62717800,영남알프스의 베이스캠프 신불산 폭포 자연휴양림
5,106,청태산 자연휴양림,(25261) 강원 횡성군 둔내면 청태산로 610,128.2907134,37.52293534,1000,63.514,56.479,75.930,17963,135846400,인공림과 천연림이 조화된 청태산 자연휴양림
6,107,삼봉 자연휴양림,(25170) 강원 홍천군 내면 삼봉휴양길 276,128.4634114,37.85798046,1500,57.750,48.099,75.401,12136,91893200,울창한 활엽수가 조화를 이룬 삼봉 자연휴양림
7,108,중미산 자연휴양림,(12505) 경기 양평군 옥천면 중미산로 1152,127.4567975,37.58092358,500,71.693,68.606,77.468,11488,80112700,중미산 자연휴양림
8,109,방태산 자연휴양림,(24656) 강원 인제군 기린면 방태산길 241,128.398925,37.92264578,600,88.415,85.027,92.908,5700,44748600,수려한 자연경관과 다양한 야생동물을 감상할 수 있는 방태산 자연휴양림
9,110,복주산 자연휴양림,(24065) 강원 철원군 근남면 하오재로 818,127.4732783,38.14381285,300,38.041,26.582,58.459,6007,42174300,울창한 산림과 맑은 계곡 복주산 자연휴양림


In [ ]:
import plotly.graph_objects as go

ratio_total   = list(df_id.loc[:,'resv_count'])
labels_total  = list(df_id.loc[:,'name'])

# pull is given as a fraction of the pie radius
fig = go.Figure(data=[go.Pie(labels=labels_total, values=ratio_total)])
fig.update_layout(margin=dict(t=20, b=20, l=10, r=20))
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

In [ ]:
fig = px.line(df.groupby(['date'],as_index=False).agg(count = ('id','count')), x="date", y="count" )
fig.show()

In [ ]:
#2021년 통계만 이용용
fig = px.bar(df[('2021-01-01'<=df.date) & (df.date<='2021-12-31')].groupby(['month'],as_index=False).agg(count = ('id','count')), x="month", y="count" , text_auto=True)
fig.show()

In [ ]:
fig = px.bar(df.groupby(['weekday'],as_index=False).agg(count = ('id','count')), x="weekday", y="count" , text_auto=True)
fig.show()

In [ ]:
df_name

In [ ]:
loc[loc.소재지도로명주소.str.contains('검마')]

In [ ]:
loc[loc.name.str.contains('검마')]

In [ ]:
loc

In [ ]:
pd.merge(df_name,loc,on='name')

In [ ]:
datalab = pd.read_csv('data/datalab.csv')
data = pd.read_csv('data/전국 캠핑장 위치 및 부가정보.csv')

In [ ]:
datalab

In [ ]:
data[data.CMPSI_NM.str.contains('평화')]